# Project 1 Edge Detection

Use the data of lab5, 2 modes of the brain MRI
- Implement the Sobel gradient kernel and get a binary mask with a specified threshold T.
- Segment the lesion (red arrow):
    - 5x5 smooth filter + Sobel filter + threshold T -> edge
    - Remove brain edge
    - Closed operation
    - Connected domain processing

Discuss
- Compare the effects of the 2 modes



# Project 2 Thresholding

Implement Iterative global threshold segmentation
- Test different initial T values

Implement Otsu's optimum thresholding algorithm to segment the lesion (Not call the library)
- Test and compare the results on the brain

# Project 3 Segmentation


- Use the T2w data
- Call Region Growing, K-mean Cluster, and Watershed Segmentation functions
- Segment the lesion and calculate its volume
    - Suppose the image resolution is (x, y)=(1mm, 1mm)

